In [ ]:
import os
import time
import numpy as np
from scipy.linalg import orth
np.random.seed(18)

In [ ]:
def generate_action_set(trials, M_list, epochs, num_target_sample, M, d, nu_list, N_i):
    
    source_data_list = []
    target_data_list = []
    
    B_star_list = []
    W_star_list = []
    Theta_star_list = []
    W_target_star_list = []
    Theta_target_star_list = []
    
    for M_index, M in enumerate(M_list):
        
        source_data = np.random.randn(epochs, trials, M, N_i, d)
        target_data = np.random.randn(epochs, trials, num_target_sample, d)
        
        source_data_list.append(source_data)
        target_data_list.append(target_data)
        
        B_star_temp = []
        W_star_temp = []
        Theta_star_temp = []
        W_target_star_temp = []
        Theta_target_star_temp = []
        
        for T in range(trials):
            
            B_star = orth(np.random.rand(d, k))
            W_star = np.random.rand(k, M)
            Theta_star = np.dot(B_star, W_star)
            W_target_star = np.dot(W_star, nu_list[M_index])
            Theta_target_star = B_star.dot(W_target_star)
            
            B_star_temp.append(B_star)
            W_star_temp.append(W_star)
            Theta_star_temp.append(Theta_star)
            W_target_star_temp.append(W_target_star)
            Theta_target_star_temp.append(Theta_target_star)
            
        B_star_list.append(B_star_temp)
        W_star_list.append(W_star_temp)
        Theta_star_list.append(Theta_star_temp)
        W_target_star_list.append(W_target_star_temp)
        Theta_target_star_list.append(Theta_target_star_temp)
        
    return source_data_list, target_data_list, B_star_list, W_star_list, Theta_star_list, W_target_star_list, Theta_target_star_list

In [ ]:
def generate_nu(M):
    
    nu = np.zeros(M)
    
    nu[:int(0.2 * M)] = 2
    nu[int(0.2 * M) : int(0.8 * M)] = 6
    nu[int(0.8 * M):] = 10
    
    np.random.shuffle(nu)
    nu = nu / np.sum(nu)
    
    return nu

In [ ]:
def init_altgdmin_1(X, Y, C, M, d, k):
    
    alpha = C * np.sum([np.sum(y ** 2) / len(y) for y in Y]) / M
    
    Y_trunc = np.copy(Y)
    Y_trunc[np.abs(Y) > alpha] = 0
    
    Theta_0 = np.zeros((d, M))
    
    for i in range(M):
        
        Theta_0[:, i] = np.sum(X[i].dot(np.diag(Y_trunc[i])), axis=1)
    
    U_0, Sigma_0, V_0 = np.linalg.svd(Theta_0, full_matrices=False)
    U_0 = U_0[:, :k]
    Sigma_0 = Sigma_0[:k]
    
    return U_0, np.max(Sigma_0)

In [ ]:
def AltGD_Min_1(X, Y, C, M, d, k, Theta_star, U_star, gd_iterations):
    
    U_init, Sigma_max = init_altgdmin_1(X, Y, C, M, d, k)
    U_hat = np.copy(U_init)
    
    B_hat = np.zeros((k, M))
    eta = 1 / Sigma_max
    
    error_list = []
    error = np.linalg.norm(U_star - U_hat, 'fro') / np.linalg.norm(U_star, 'fro')
    error_list.append(error)
    
    for num in range(gd_iterations):
        
        for i in range(M):
            
            B_hat[:, i] = np.linalg.pinv(np.dot(X[i].T, U_hat)).dot(Y[i])
            
        Theta_hat = np.dot(U_hat, B_hat)
        U_grad = np.zeros((d, k))
        
        for i in range(M):
            
            U_grad += X[i] @ (X[i].T @ Theta_hat[:, i].reshape(-1, 1) - Y[i].reshape(-1, 1)) @ B_hat[:, i].reshape(1, -1) / X[i].shape[-1]
            
        U_hat = U_hat - eta * U_grad
        q, r = np.linalg.qr(U_hat)
        U_hat = q[:, :k]
        
        Theta_hat_ = np.dot(U_hat, B_hat)
        error = np.linalg.norm(Theta_star - Theta_hat_, 'fro') / np.linalg.norm(Theta_star, 'fro')
        error_list.append(error)
        
    return U_hat, B_hat, eta, error_list

In [ ]:
def AltGD_Min_2(X, Y, M, d, k, B_hat, eta, gd_iterations):
    
    U_hat = np.copy(B_hat)
    
    B_hat = np.zeros((k, M))
    
    for num in range(gd_iterations):
        
        for i in range(M):
            
            B_hat[:, i] = np.linalg.pinv(np.dot(X[i].T, U_hat)).dot(Y[i])
            
        Theta_hat = np.dot(U_hat, B_hat)
            
        U_grad = np.zeros((d, k))
        
        for i in range(M):
            
            U_grad += X[i] @ (X[i].T @ Theta_hat[:, i].reshape(-1, 1) - Y[i].reshape(-1, 1)) @ B_hat[:, i].reshape(1, -1) / X[i].shape[-1]
            
        U_hat = U_hat - eta * U_grad
        q, r = np.linalg.qr(U_hat)
        U_hat = q[:, :k]
        
    return U_hat, B_hat

In [ ]:
def gd_Theta(X, Y, M, gd_iteration, learning_rate, Theta_star):
    
    def compute_loss_grad(X, Y, Theta, M):
        
        loss = 0
        grad = np.zeros_like(Theta)
        
        for m in range(M):
            
            error_list = X[m].T.dot(Theta[:, m]) - Y[m]
            loss += 0.5 * np.sum(error_list ** 2)
            grad[:, m] = X[m].dot(error_list)
            
        nuclear_norm = np.sum(np.linalg.svd(Theta, compute_uv=False))
        loss += nuclear_norm
        
        U, _, Vt = np.linalg.svd(Theta, full_matrices=False)
        nuclear_grad = np.dot(U, Vt)
        grad += nuclear_grad
            
        return loss, grad
    
    Theta = np.ones((d, M))
    error_list = []
    
    error = np.linalg.norm(Theta_star - Theta, 'fro') / np.linalg.norm(Theta_star, 'fro')
    error_list.append(error)
    
    for i in range(gd_iteration):
        
        loss, grad = compute_loss_grad(X, Y, Theta, M)
        Theta -= learning_rate * grad
        
        error = np.linalg.norm(Theta_star - Theta, 'fro') / np.linalg.norm(Theta_star, 'fro')
        error_list.append(error)
        
    return Theta, error_list

In [ ]:
def init_altgdmin_2(X, Y, M, d, k):
    
    Z = np.zeros((d, d))
    
    for m in range(M):
        
        for n in range(X[m].shape[1]):
            
            Z += Y[m][n] ** 2 * np.dot(X[m][:, n][:, np.newaxis], X[m][:, n][:, np.newaxis].T)
            
        Z /= X[m].shape[1]
        
    Z /= m
    
    U_0, Sigma_0, V_0 = np.linalg.svd(Z, full_matrices=False)
    U_0 = U_0[:, :k]
    
    return U_0

In [ ]:
def AltGD_Min_Collins1(X, Y, M, d, k, Theta_star, U_star, gd_iterations):
    
    U_init = init_altgdmin_2(X, Y, M, d, k)
    U_hat = np.copy(U_init)
    B_hat = np.zeros((k, M))
    
    error_list = []
    error = np.linalg.norm(U_star - U_hat, 'fro') / np.linalg.norm(U_star, 'fro')
    error_list.append(error)
    
    for num in range(gd_iterations):
        
        M_sub_value = int(np.random.uniform(low = 1e-1, high = 1 + 1e-10) * M)
        M_sub = np.random.choice(range(M), M_sub_value, replace = False)
        
        for i in M_sub:
            
            XB = X[i].T.dot(U_hat)
            B_hat[:, i] = np.linalg.pinv(XB.T.dot(XB)).dot(XB.T).dot(Y[i])
            
        Theta_hat = np.dot(U_hat, B_hat)
        U_grad = np.zeros((d, k))
        
        for i in M_sub:
            
            U_grad += X[i] @ (X[i].T @ Theta_hat[:, i].reshape(-1, 1) - Y[i].reshape(-1, 1)) @ B_hat[:, i].reshape(1, -1) / X[i].shape[-1]
            
        U_hat = U_hat - (1 / M_sub_value) * U_grad
        # q, r = np.linalg.qr(U_hat)
        # U_hat = q[:, :k]
        
        Theta_hat_ = np.dot(U_hat, B_hat)
        error = np.linalg.norm(Theta_star - Theta_hat_, 'fro') / np.linalg.norm(Theta_star, 'fro')
        error_list.append(error)
        
    return U_hat, B_hat, error_list

In [ ]:
def AltGD_Min_Collins2(X, Y, M, d, k, B_hat, gd_iterations):
    
    U_hat = np.copy(B_hat)
    
    B_hat = np.zeros((k, M))
    
    for num in range(gd_iterations):
        
        M_sub_value = int(np.random.uniform(low = 1e-1, high = 1 + 1e-10) * M)
        M_sub = np.random.choice(range(M), M_sub_value, replace = False)
        
        for i in M_sub:
            
            XB = X[i].T.dot(U_hat)
            B_hat[:, i] = np.linalg.pinv(XB.T.dot(XB)).dot(XB.T).dot(Y[i])
            
        Theta_hat = np.dot(U_hat, B_hat)
            
        U_grad = np.zeros((d, k))
        
        for i in M_sub:
            
            U_grad += X[i] @ (X[i].T @ Theta_hat[:, i].reshape(-1, 1) - Y[i].reshape(-1, 1)) @ B_hat[:, i].reshape(1, -1) / X[i].shape[-1]
            
        U_hat = U_hat - (1 / M_sub_value) * U_grad
        q, r = np.linalg.qr(U_hat)
        U_hat = q[:, :k]
        
    return U_hat, B_hat

In [ ]:
# Setting parameters
C = [1.0, 1.2, 1.6]
k = 2
d = 100
ld_0 = 1
epochs = 4
trials = 100
delta = 1e-3
noise_var = 1e-2
learning_rate = 1e-3
gd_iterations = [40, 40, 40]
num_target_sample = 30
M_list = [40, 60, 80]
gd_iterations_chen = 50
increase_gd_iteration = [10, 10, 10]
ER_list1 = []
ER_list2 = []
ER_list3 = []
ER_list4 = []
ER_list5 = []
gd_error_list1 = []
gd_error_list3 = []
gd_error_list4 = []
gd_error_list5 = []
percentage_error_list2 = []

In [ ]:
# generate the nu
nu_list = []

for M_index, M in enumerate(M_list):
    
#     # method 1
#     nu = np.random.rand(M)
#     nu = np.power(nu, 3)
#     nu /= np.sum(nu)
    
    # method 2
    nu = generate_nu(M)
    
    nu_list.append(nu)

In [ ]:
# set the total number of sample
num_sample = 50
N_i = num_sample * M
N = N_i * epochs

In [ ]:
# generate the data
source_data_list, target_data_list, B_star_list, W_star_list, Theta_star_list, W_target_star_list, Theta_target_star_list = generate_action_set(trials, M_list, epochs, num_target_sample, M, d, nu_list, N_i)

In [ ]:
# generate the noise
noise = np.random.normal(0, noise_var, size = (trials, epochs, M + 1, N_i))

In [ ]:
# AltGDmin Algorithm
start = time.time()

for T in range(trials):
    
    print('Trial:', T)
    
    for index, M in enumerate(M_list):
        
        B_star = B_star_list[index]
        W_star = W_star_list[index]
        Theta_star = Theta_star_list[index]
        W_target_star = W_target_star_list[index]
        Theta_target_star = Theta_target_star_list[index]
        
        ER_1 = []
        nu_hat = [1 / M for m in range(M)]
        
        for i in range(epochs):
            
            source_data = source_data_list[index][i]
            target_data = target_data_list[index][i]
            
            X = []
            Y = []
            
            # new
            index_m = []
            
            for m in range(M):
                
                n_m = int((N / epochs) * (np.abs(nu_hat) / np.linalg.norm(nu_hat, 1))[m])
                
                # new
                if n_m == 0:
                    
                    M = M - 1
                    index_m.append(m)
                    
                    continue
                
                X_temp = np.zeros((d, n_m))
                Y_temp = np.zeros(n_m)
                
                for num in range(n_m):
                    
                    X_temp[:, num] = source_data[T][m][num]
                    Y_temp[num] = source_data[T][m][num].dot(Theta_star[T][:, m]) + noise[T][i][m][num]
                    
                X.append(X_temp)
                Y.append(Y_temp)
                
            # new
            if len(index_m) > 0:
                
                W_hat_m = W_hat[:, index_m]
                
            if i == 0:
                
                B_hat, W_hat, eta, error_list1 = AltGD_Min_1(X, Y, C[index], M, d, k, Theta_star[T], B_star[T], gd_iterations[index])
                
            else:
                
                B_hat, W_hat = AltGD_Min_2(X, Y, M, d, k, B_hat, eta, gd_iterations[index] + (i - 1) * increase_gd_iteration[index])
                
            # new
            if len(index_m) > 0:
                
                j = 0
                W_temp = np.zeros((k, M + len(index_m)))
                
                for v in range(M + len(index_m)):
                    
                    if v in index_m:
                        
                        continue
                        
                    W_temp[:, v] = W_hat[:, j]
                    j += 1
                    
                for idx, column in zip(index_m, W_hat_m.T):
                    
                    W_temp[:, idx] = column
                    
                W_hat = W_temp
                M = M_list[index]
                
            Theta_hat = B_hat.dot(W_hat)
            error = np.linalg.norm(Theta_star[T] - Theta_hat, 'fro') / np.linalg.norm(Theta_star[T], 'fro')
            
            X_target = np.zeros((num_target_sample, d))
            Y_target = np.zeros(num_target_sample)
            
            for num in range(num_target_sample):
                
                X_target[num, :] = target_data[T][num]
                Y_target[num] = target_data[T][num].dot(Theta_target_star[T]) + noise[T][i][M][num]
                
            w_hat_target = np.linalg.pinv(X_target.dot(B_hat)).dot(Y_target)
            nu_hat = W_hat.T.dot(np.linalg.pinv(np.dot(W_hat, W_hat.T))).dot(w_hat_target)
            nu_error = np.linalg.norm(nu_hat - np.linalg.pinv(W_star[T]).dot(W_target_star[T])) / np.linalg.norm(nu)
            
            term1 = X_target.dot(B_hat).dot(w_hat_target)
            term2 = X_target.dot(Theta_target_star[T])
            ER = np.sum((term1 - term2) ** 2) / num_target_sample
            
            ER_1.append(ER)
            # print('M: {} epoch: {} error: {} nu_error: {}:'.format(M, i, error, nu_error))
            
        ER_list1.append((T, M, ER_1))
        gd_error_list1.append((T, M, error_list1))
        
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
# MOM Algorithm
start = time.time()

for T in range(trials):
    
    print('Trial:', T)
    
    for index, M in enumerate(M_list):
        
        B_star = B_star_list[index]
        W_star = W_star_list[index]
        Theta_star = Theta_star_list[index]
        W_target_star = W_target_star_list[index]
        Theta_target_star = Theta_target_star_list[index]
        
        ER_2 = []
        percentage_error_2 = []
        B_hat = np.zeros((d, k))
        W_hat = np.zeros((k, M))
        Theta_hat = B_hat.dot(W_hat)
        
        M_hat = np.zeros((d, d))
        nu_hat = [1 / M for m in range(M)]
        
        for i in range(epochs):
            
            source_data = source_data_list[index][i]
            target_data = target_data_list[index][i]
            
            if i == 0:
                
                n_m_i = 0
                
                for m in range(M):
                    
                    n_m = int((N / epochs) * (np.abs(nu_hat) / np.linalg.norm(nu_hat, 1))[m])
                    n_m_i += n_m
                    
                    for num in range(n_m):
                        
                        x_temp = source_data[T][m][num]
                        y_temp = source_data[T][m][num].dot(Theta_star[T][:, m]) + noise[T][i][m][num]
                        M_hat = M_hat + y_temp ** 2 * np.dot(x_temp.T, x_temp)
                        
                M_hat = (1 / n_m_i) * M_hat
                
                U, Sigma, V = np.linalg.svd(M_hat, full_matrices=False)
                B_hat = U[:, :k]
            
                X_target = np.zeros((num_target_sample, d))
                Y_target = np.zeros(num_target_sample)
                
                for num in range(num_target_sample):

                    X_target[num, :] = target_data[T][num]
                    Y_target[num] = target_data[T][num].dot(Theta_target_star[T]) + noise[T][i][M][num]
                    
                w_hat_target = np.linalg.pinv(X_target.dot(B_hat)).dot(Y_target)
                
                term1 = X_target.dot(B_hat).dot(w_hat_target)
                term2 = X_target.dot(Theta_target_star[T])
                ER = np.sum((term1 - term2) ** 2) / num_target_sample
                ER_2.append(ER)
                
            elif i == 1:
                
                X = []
                Y = []
                
                for m in range(M):
                    
                    n_m = int((N / epochs) * (np.abs(nu_hat) / np.linalg.norm(nu_hat, 1))[m])
                    
                    X_temp = np.zeros((d, int(n_m / k) * k))
                    Y_temp = np.zeros(int(n_m / k) * k)
                    
                    for num in range(int(n_m / k)):
                        
                        for k_value in range(k):
                            
                            X_temp[:, num * k + k_value] = np.sqrt(ld_0) * B_hat[:, k_value]
                            Y_temp[num * k + k_value] = X_temp[:, num * k + k_value].reshape(1, -1).dot(Theta_star[T][:, m]) + noise[T][i][m][num]
                            
                    X.append(X_temp)
                    Y.append(Y_temp)
                    
                for m in range(M):
                    
                    temp = X[m].T.dot(B_hat)
                    W_hat[:, m] = np.linalg.lstsq(temp, Y[m], rcond=None)[0]
                    
                Theta_hat = B_hat.dot(W_hat)
                
                error = np.linalg.norm(Theta_star[T] - Theta_hat, 'fro') / np.linalg.norm(Theta_star[T], 'fro')
                
                X_target = np.zeros((num_target_sample, d))
                Y_target = np.zeros(num_target_sample)
                
                for num in range(num_target_sample):
                    
                    X_target[num, :] = target_data[T][num]
                    Y_target[num] = target_data[T][num].dot(Theta_target_star[T]) + noise[T][i][M][num]
                    
                w_hat_target = np.linalg.pinv(X_target.dot(B_hat)).dot(Y_target)
                nu_hat = W_hat.T.dot(np.linalg.pinv(np.dot(W_hat, W_hat.T))).dot(w_hat_target)
                nu_error = np.linalg.norm(nu_hat - np.linalg.pinv(W_star[T]).dot(W_target_star[T])) / np.linalg.norm(nu)
                
                term1 = X_target.dot(B_hat).dot(w_hat_target)
                term2 = X_target.dot(Theta_target_star[T])
                ER = np.sum((term1 - term2) ** 2) / num_target_sample
                
                ER_2.append(ER)
                percentage_error_2.append(error)
                # print('M: {} epoch: {} error: {} nu_error: {}:'.format(M, i, error, nu_error))
                
            else:
                
                X_target = np.zeros((num_target_sample, d))
                Y_target = np.zeros(num_target_sample)
                
                for num in range(num_target_sample):
                    
                    X_target[num, :] = target_data[T][num]
                    Y_target[num] = target_data[T][num].dot(Theta_target_star[T]) + noise[T][i][M][num]
                    
                w_hat_target = np.linalg.pinv(X_target.dot(B_hat)).dot(Y_target)
                nu_hat = W_hat.T.dot(np.linalg.pinv(np.dot(W_hat, W_hat.T))).dot(w_hat_target)
                nu_error = np.linalg.norm(nu_hat - np.linalg.pinv(W_star[T]).dot(W_target_star[T])) / np.linalg.norm(nu)
                
                term1 = X_target.dot(B_hat).dot(w_hat_target)
                term2 = X_target.dot(Theta_target_star[T])
                ER = np.sum((term1 - term2) ** 2) / num_target_sample
                
                ER_2.append(ER)
                # print('M: {} epoch: {} error: {} nu_error: {}:'.format(M, i, error, nu_error))
            
        ER_list2.append((T, M, ER_2))
        percentage_error_list2.append((T, M, percentage_error_2))
        
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
# Chen et al. Algorithm
start = time.time()

for T in range(trials):
    
    print('Trial:', T)
    
    for index, M in enumerate(M_list):
        
        B_star = B_star_list[index]
        W_star = W_star_list[index]
        Theta_star = Theta_star_list[index]
        W_target_star = W_target_star_list[index]
        Theta_target_star = Theta_target_star_list[index]
        
        ER_3 = []
        nu_hat = [1 / M for m in range(M)]
        
        for i in range(epochs):
            
            source_data = source_data_list[index][i]
            target_data = target_data_list[index][i]
            
            X = []
            Y = []
            
            for m in range(M):
                
                n_m = int((N / epochs) * (np.abs(nu_hat) / np.linalg.norm(nu_hat, 1))[m])
                
                X_temp = np.zeros((d, n_m))
                Y_temp = np.zeros(n_m)
                
                for num in range(n_m):
                    
                    X_temp[:, num] = source_data[T][m][num]
                    Y_temp[num] = source_data[T][m][num].dot(Theta_star[T][:, m]) + noise[T][i][m][num]
                    
                X.append(X_temp)
                Y.append(Y_temp)
                
            Theta_hat, error_list_chen = gd_Theta(X, Y, M, gd_iterations_chen, learning_rate, Theta_star[T])
            B_hat, Sigma_hat, V_hat = np.linalg.svd(Theta_hat, full_matrices=False)
            B_hat = B_hat[:, :k]
            W_hat = np.diag(Sigma_hat[:k]).dot(V_hat[:k, :])
            
            error = np.linalg.norm(Theta_star[T] - Theta_hat, 'fro') / np.linalg.norm(Theta_star[T], 'fro')
            
            X_target = np.zeros((num_target_sample, d))
            Y_target = np.zeros(num_target_sample)
            
            for num in range(num_target_sample):
                
                X_target[num, :] = target_data[T][num]
                Y_target[num] = target_data[T][num].dot(Theta_target_star[T]) + noise[T][i][M][num]
                
            w_hat_target = np.linalg.pinv(X_target.dot(B_hat)).dot(Y_target)
            nu_hat = W_hat.T.dot(np.linalg.pinv(np.dot(W_hat, W_hat.T))).dot(w_hat_target)
            nu_error = np.linalg.norm(nu_hat - np.linalg.pinv(W_star[T]).dot(W_target_star[T])) / np.linalg.norm(nu)
            
            term1 = X_target.dot(B_hat).dot(w_hat_target)
            term2 = X_target.dot(Theta_target_star[T])
            ER = np.sum((term1 - term2) ** 2) / num_target_sample
            
            ER_3.append(ER)
            # print('M: {} epoch: {} error: {} nu_error: {}:'.format(M, i, error, nu_error))aa'aa
            
        ER_list3.append((T, M, ER_3))
        gd_error_list3.append((T, M, error_list_chen))
        
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
# Uniform Sampling
start = time.time()

for T in range(trials):
    
    print('Trial:', T)
    
    for index, M in enumerate(M_list):
        
        B_star = B_star_list[index]
        W_star = W_star_list[index]
        Theta_star = Theta_star_list[index]
        W_target_star = W_target_star_list[index]
        Theta_target_star = Theta_target_star_list[index]
        
        ER_4 = []
        nu_hat = [1 / M for m in range(M)]
        
        for i in range(epochs):
            
            source_data = source_data_list[index][i]
            target_data = target_data_list[index][i]
            
            X = []
            Y = []
            
            for m in range(M):
                
                n_m = int((N / epochs) * (np.abs([1 / M for m in range(M)]) / np.linalg.norm([1 / M for m in range(M)], 1))[m])
                
                X_temp = np.zeros((d, n_m))
                Y_temp = np.zeros(n_m)
                
                for num in range(n_m):
                    
                    X_temp[:, num] = source_data[T][m][num]
                    Y_temp[num] = source_data[T][m][num].dot(Theta_star[T][:, m]) + noise[T][i][m][num]
                    
                X.append(X_temp)
                Y.append(Y_temp)
                
            if i == 0:
                
                B_hat, W_hat, eta, error_list4 = AltGD_Min_1(X, Y, C[index], M, d, k, Theta_star[T], B_star[T], gd_iterations[index])
                
            else:
                
                B_hat, W_hat = AltGD_Min_2(X, Y, M, d, k, B_hat, eta, gd_iterations[index] + (i - 1) * increase_gd_iteration[index])
                
            Theta_hat = B_hat.dot(W_hat)
            error = np.linalg.norm(Theta_star[T] - Theta_hat, 'fro') / np.linalg.norm(Theta_star[T], 'fro')
            
            X_target = np.zeros((num_target_sample, d))
            Y_target = np.zeros(num_target_sample)
            
            for num in range(num_target_sample):
                
                X_target[num, :] = target_data[T][num]
                Y_target[num] = target_data[T][num].dot(Theta_target_star[T]) + noise[T][i][M][num]
                
            w_hat_target = np.linalg.pinv(X_target.dot(B_hat)).dot(Y_target)
            nu_hat = W_hat.T.dot(np.linalg.pinv(np.dot(W_hat, W_hat.T))).dot(w_hat_target)
            nu_error = np.linalg.norm(nu_hat - np.linalg.pinv(W_star[T]).dot(W_target_star[T])) / np.linalg.norm(nu)
            
            term1 = X_target.dot(B_hat).dot(w_hat_target)
            term2 = X_target.dot(Theta_target_star[T])
            ER = np.sum((term1 - term2) ** 2) / num_target_sample
            
            ER_4.append(ER)
            # print('M: {} epoch: {} error: {} nu_error: {}:'.format(M, i, error, nu_error))
            
        ER_list4.append((T, M, ER_4))
        gd_error_list4.append((T, M, error_list4))
        
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
# Collins Algorithm
start = time.time()

for T in range(trials):
    
    print('Trial:', T)
    
    for index, M in enumerate(M_list):
        
        B_star = B_star_list[index]
        W_star = W_star_list[index]
        Theta_star = Theta_star_list[index]
        W_target_star = W_target_star_list[index]
        Theta_target_star = Theta_target_star_list[index]
        
        ER_5 = []
        nu_hat = [1 / M for m in range(M)]
        
        for i in range(epochs):
            
            source_data = source_data_list[index][i]
            target_data = target_data_list[index][i]
            
            X = []
            Y = []
            
            # new
            index_m = []
            
            for m in range(M):
                
                n_m = int((N / epochs) * (np.abs(nu_hat) / np.linalg.norm(nu_hat, 1))[m])
                
                # new
                if n_m == 0:
                    
                    M = M - 1
                    index_m.append(m)
                    
                    continue
                    
                X_temp = np.zeros((d, n_m))
                Y_temp = np.zeros(n_m)
                
                for num in range(n_m):
                    
                    X_temp[:, num] = source_data[T][m][num]
                    Y_temp[num] = source_data[T][m][num].dot(Theta_star[T][:, m]) + noise[T][i][m][num]
                    
                X.append(X_temp)
                Y.append(Y_temp)
                
            # new
            if len(index_m) > 0:
                
                W_hat_m = W_hat[:, index_m]
                
            if i == 0:
                
                B_hat, W_hat, error_list5 = AltGD_Min_Collins1(X, Y, M, d, k, Theta_star[T], B_star[T], gd_iterations[index])
                
            else:
                
                B_hat, W_hat = AltGD_Min_Collins2(X, Y, M, d, k, B_hat, gd_iterations[index] + (i - 1) * increase_gd_iteration[index])
                
            # new
            if len(index_m) > 0:
                
                j = 0
                W_temp = np.zeros((k, M + len(index_m)))
                
                for v in range(M + len(index_m)):
                    
                    if v in index_m:
                        
                        continue
                        
                    W_temp[:, v] = W_hat[:, j]
                    j += 1
                    
                for idx, column in zip(index_m, W_hat_m.T):
                    
                    W_temp[:, idx] = column
                    
                W_hat = W_temp
                M = M_list[index]
                
            Theta_hat = B_hat.dot(W_hat)
            error = np.linalg.norm(Theta_star[T] - Theta_hat, 'fro') / np.linalg.norm(Theta_star[T], 'fro')
            
            X_target = np.zeros((num_target_sample, d))
            Y_target = np.zeros(num_target_sample)
            
            for num in range(num_target_sample):
                
                X_target[num, :] = target_data[T][num]
                Y_target[num] = target_data[T][num].dot(Theta_target_star[T]) + noise[T][i][M][num]
                
            w_hat_target = np.linalg.pinv(X_target.dot(B_hat)).dot(Y_target)
            nu_hat = W_hat.T.dot(np.linalg.pinv(np.dot(W_hat, W_hat.T))).dot(w_hat_target)
            nu_error = np.linalg.norm(nu_hat - np.linalg.pinv(W_star[T]).dot(W_target_star[T])) / np.linalg.norm(nu)
            
            term1 = X_target.dot(B_hat).dot(w_hat_target)
            term2 = X_target.dot(Theta_target_star[T])
            ER = np.sum((term1 - term2) ** 2) / num_target_sample
            
            ER_5.append(ER)
            # print('M: {} epoch: {} error: {} nu_error: {}:'.format(M, i, error, nu_error))
            
        ER_list5.append((T, M, ER_5))
        gd_error_list5.append((T, M, error_list5))
        
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
np.save('ER_list1.npy', np.array(ER_list1, dtype=object))
np.save('ER_list2.npy', np.array(ER_list2, dtype=object))
np.save('ER_list3.npy', np.array(ER_list3, dtype=object))
np.save('ER_list4.npy', np.array(ER_list4, dtype=object))
np.save('ER_list5.npy', np.array(ER_list5, dtype=object))
np.save('gd_error_list1.npy', np.array(gd_error_list1, dtype=object))
np.save('gd_error_list3.npy', np.array(gd_error_list3, dtype=object))
np.save('gd_error_list4.npy', np.array(gd_error_list4, dtype=object))
np.save('gd_error_list5.npy', np.array(gd_error_list5, dtype=object))
np.save('percentage_error_list2.npy', np.array(percentage_error_list2, dtype=object))